# Clustering process for policies

This code permits the culstering of policies.

It uses Sentence BERT as an embedder and HDBSCAN as a clustering algorithm. 
- Sentence BERT is a Python module for accessing, using, and training state-of-the-art text and image embedding models. It can be used to compute embeddings using Sentence Transformer models (quickstart) or to calculate similarity scores using Cross-Encoder models.
- HDBSCAN is a clustering algorithm extending DBSCAN by converting it into a hierarchical clustering algorithm. DBSCAN is a density-based clustering method that finds core samples and expands clusters from them. 

In this Jupyter Notebook we will: 
1. Import the data retrieved from the policy extraction process ; 
2. Import the relevant packages ;
3. Prepare data for clustering ;
4. Cluster with HDBSCAN ; 
5. Re-process the clusters ; 
    1. Export for manual check ;
    2. Reclustering with HDBSCAN ;
    3. Clean and name the final clusters ;
6. Export the policy clustering data. 

To complete those tasks you will need:
- The dataset of papers with the policy extraction of 1_policy_extraction code. 

At the end of this script you will extract: 
- The named_cluster_df dataset of policy clusters and the sentences extracted during the 1_policy_extraction. 

## 1. Import the data retrieved from the screening process

Change the input and output access paths:

In [2]:
# Output dataset of the 1_policy_extraction
input_path = ""

# 3 outputs are needed
## First to export the first HDBSCAN results to check by hand
output_path_first_manual_check = ""
## Second checking after reprocessing
output_path_second_manual_check = ""
## Final extraction
export_path = ""

## 2. Import the relevant packages

In [ ]:
import pandas as pd
import json
import re
import random

In [1]:
# Packages for HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import HDBSCAN
from joblib import Parallel, delayed
import numpy as np

c:\Users\easycash\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## 3. Prepare data for clustering

In [ ]:
# Import data 
df = pd.read_csv(input_path)

In [ ]:
# Function to clean and parse JSON
def clean_and_parse_json(json_string):
    try:
        # Skip invalid strings
        if not json_string.strip().startswith("{"):
            return None
        # Remove trailing commas
        cleaned_string = re.sub(r",\s*}", "}", json_string)
        cleaned_string = re.sub(r",\s*]", "]", cleaned_string)
        # Parse the cleaned JSON string
        return json.loads(cleaned_string)
    except json.JSONDecodeError:
        return None

In [ ]:
# Function to check if the JSON is meaningful
def is_meaningful_json(parsed_data):
    if not isinstance(parsed_data, dict):
        return False
    # Check if the JSON only contains "None" values
    for key, value in parsed_data.items():
        if key != "None" or (isinstance(value, dict) and any(k != "None" for k in value.keys())):
            return True
    return False

In [ ]:
def extract_items(dataframe):
    new_rows = []
    for idx, row in dataframe.iterrows():
        extracted_data = row['extracted_features_and_correlations']
        
        # Skip rows with "No abstract" or invalid data
        if extracted_data == "No abstract" or not isinstance(extracted_data, str):
            continue
        
        # Clean and parse JSON content
        parsed_data = clean_and_parse_json(extracted_data)
        if parsed_data is None or not is_meaningful_json(parsed_data):
            print(f"Skipping non-meaningful JSON for index {idx}")
            continue
        
        geographic = parsed_data.get("GEOGRAPHIC", "None")
        
        # Iterate through the items
        for item, details in parsed_data.items():
            if item == "GEOGRAPHIC":  # Skip the geographic key
                continue
            
            # Ensure details is a dictionary before accessing keys
            if not isinstance(details, dict):
                print(f"Skipping invalid details for item {item} at index {idx}: {details}")
                continue

            actor = details.get("ACTOR", "None")
            mode = details.get("MODE", "None")
            population = details.get("POPULATION", "None")
            
            # Append new row for each ITEM
            new_rows.append({
                'index': idx,  # Use original index as reference
                'GEOGRAPHIC': geographic,
                'ITEM': item,
                'ACTOR': actor,
                'MODE': mode,
                'POPULATION': population
            })
    
    # Create new DataFrame
    new_df = pd.DataFrame(new_rows)

    # Create a new index with suffix for duplicates
    new_df['new_index'] = new_df.groupby('index').cumcount().add(1).astype(str)
    new_df['new_index'] = new_df['index'].astype(str) + "_" + new_df['new_index']
    return new_df

In [ ]:
# Apply the extraction
extracted_items_df = extract_items(df)

In [ ]:
def concatenate_columns(df):
    # Define a function to concatenate values if they are not "None"
    def concatenate(row):
        values = [row['ITEM'], row['MODE']]
        # Filter out "None" values and join with a space
        return " ".join(str(value) for value in values if value != "None")
    
    # Apply the function to each row and create a new column
    df['concatenated_column'] = df.apply(concatenate, axis=1)
    return df

# Apply the function to the extracted_items_df
updated_df = concatenate_columns(extracted_items_df)

## 4. Cluster with HDBSCAN

In [ ]:
preprocessed_corpus = updated_df['concatenated_column']

In [ ]:
# Step 1: Initialize Smaller Model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2: Ensure Preprocessed Corpus has a Continuous Index
preprocessed_corpus = preprocessed_corpus.reset_index(drop=True)

# Convert to a list for parallel processing
corpus_list = preprocessed_corpus.tolist()

# Step 3: Batch Embedding Function
def embed_batch(batch):
    return embedder.encode(batch, show_progress_bar=False)

# Step 4: Generate Embeddings in Batches (Parallelized)
def parallel_embedding(corpus, batch_size=512):
    embeddings = Parallel(n_jobs=-1)(
        delayed(embed_batch)(corpus[i:i + batch_size])
        for i in range(0, len(corpus), batch_size)
    )
    return np.vstack(embeddings)

# Encode the corpus in parallel
batch_size = 512
corpus_embeddings = parallel_embedding(corpus_list, batch_size=batch_size)

# Step 5: Apply Dimensionality Reduction (PCA) Before Normalization
pca = PCA(n_components=50, random_state=42)
reduced_embeddings = pca.fit_transform(corpus_embeddings)
reduced_embeddings = normalize(reduced_embeddings)

# Step 6: Apply HDBSCAN Clustering
# HDBSCAN automatically determines the number of clusters
hdbscan_model = HDBSCAN(
    min_cluster_size=50,  # Minimum cluster size
    min_samples=10,        # Minimum samples in a neighborhood for a core point
    metric='euclidean',   # Distance metric
    cluster_selection_epsilon=0.5  # Adjust for fine-grained clustering
)
cluster_assignment = hdbscan_model.fit_predict(reduced_embeddings)

# Step 7: Analyze and Visualize Clusters
# HDBSCAN assigns -1 to noise points
num_clusters_found = len(set(cluster_assignment)) - (1 if -1 in cluster_assignment else 0)
print(f"Number of clusters found: {num_clusters_found}")

# Group sentences by cluster
clustered_sentences = [[] for _ in range(num_clusters_found)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id != -1:  # Exclude noise points
        clustered_sentences[cluster_id].append(corpus_list[sentence_id])

# Print clusters
for i, cluster in enumerate(clustered_sentences):
    print(f"Cluster {i + 1}:")
    print(cluster)
    print("")

## 5. Re-process the clusters

### 1. Export for manual check

In [ ]:
## Extraction of a random sample of sentences to validation

# Suppress duplicate sentences
unique_clustered_sentences = [
    list(set(cluster)) for cluster in clustered_sentences
]

# Create the DataFrame
data = []
for cluster_num, sentences in enumerate(unique_clustered_sentences):
    # Get the number of sentences in the cluster
    num_sentences = len(sentences)
    
    # Randomly sample 15 sentences (or fewer if the cluster has less than 10 sentences)
    sample_sentences = random.sample(sentences, min(15, num_sentences))
    
    # Append the cluster info to the data list
    data.append({
        "Cluster Number": cluster_num + 1,
        "Number of Sentences": num_sentences,
        "Sample Sentences": "; ".join(sample_sentences)
    })

# Convert to DataFrame
cluster_summary_df = pd.DataFrame(data)

In [ ]:
# Export the clusters for manual check
cluster_summary_df.to_csv(output_path_first_manual_check, index=False)

### 2. Reclustering with HDBSCAN

After choosing the clusters to subdivide (here clusters 6 and 11), use HDBSCAN to recluster. 

In [ ]:
## Re-clsutering of subdivision of clusters
### List of clusters to subdivide
clusters_to_subdivide = [6, 11]

# Function to subdivide clusters and create a new dataframe
def subdivide_clusters_to_new_dataframe(clustered_sentences, cluster_assignment, reduced_embeddings, clusters_to_subdivide):
    # Create a list for the final combined clusters
    combined_clusters = []
    new_subclusters = []  # To hold subdivided clusters

    # Add clusters that are not being subdivided to the final list
    for cluster_id, sentences in enumerate(clustered_sentences):
        if (cluster_id + 1) not in clusters_to_subdivide:  # Adjust for 1-based indexing in `clusters_to_subdivide`
            combined_clusters.append({"cluster_id": cluster_id + 1, "sentences": sentences})

    # Subdivide the specified clusters
    for cluster_index in clusters_to_subdivide:
        # Adjust index for 0-based indexing (Python lists)
        cluster_id = cluster_index - 1

        # Extract the embeddings and sentences for the current cluster
        indices = [i for i, cid in enumerate(cluster_assignment) if cid == cluster_id]
        if len(indices) < 5:  # HDBSCAN needs at least a few points
            continue

        cluster_embeddings = reduced_embeddings[indices]
        cluster_sentences = [preprocessed_corpus[i] for i in indices]

        # Apply HDBSCAN to subdivide the cluster
        # Apply HDBSCAN to subdivide the cluster
        hdbscan_model = HDBSCAN(
            min_cluster_size=5,  # Minimum cluster size
            min_samples=5,        # Minimum samples in a neighborhood for a core point
            metric='euclidean',   # Distance metric
            cluster_selection_epsilon=0.45  # Adjust for fine-grained clustering
            )
        hdbscan_labels = hdbscan_model.fit_predict(cluster_embeddings)

        # Map each HDBSCAN cluster to the combined list
        for hdbscan_cluster_id in set(hdbscan_labels):
            if hdbscan_cluster_id == -1:  # Skip noise
                continue
            new_subclusters.append(
                {
                    "cluster_id": f"{cluster_index}-{hdbscan_cluster_id}",
                    "sentences": [cluster_sentences[i] for i, label in enumerate(hdbscan_labels) if label == hdbscan_cluster_id],
                }
            )

    # Append subdivided clusters to the remaining clusters
    combined_clusters.extend(new_subclusters)

    # Convert the combined clusters into a dataframe
    new_cluster_df = pd.DataFrame(combined_clusters)
    return new_cluster_df

# Subdivide selected clusters and create a new dataframe
new_cluster_df = subdivide_clusters_to_new_dataframe(
    clustered_sentences, cluster_assignment, reduced_embeddings, clusters_to_subdivide
)

# Display the new dataframe
print(new_cluster_df)

In [ ]:
# Update with your desired output path
new_cluster_df.to_csv(output_path_second_manual_check, index=False)

### 3. Clean and name the final clusters

Clean by suppressing the clusters off topic, merge and finally name the clusters. 

In [ ]:
def suppress_and_merge_clusters(cluster_df, clusters_to_suppress, clusters_to_merge):
    """
    Suppress and merge clusters in a DataFrame.

    Parameters:
    - cluster_df: DataFrame containing cluster information.
    - clusters_to_suppress: List of cluster IDs to suppress.
    - clusters_to_merge: Dictionary where keys are clusters to keep, and values are lists of clusters to merge into them.

    Returns:
    - Updated DataFrame with suppressed and merged clusters.
    """
    # Suppress clusters
    suppressed_df = cluster_df[~cluster_df["cluster_id"].isin(clusters_to_suppress)]

    # Merge clusters
    for target_cluster, clusters_to_merge_into in clusters_to_merge.items():
        # Find the sentences for all clusters to merge
        sentences_to_merge = []
        for merge_cluster in clusters_to_merge_into:
            merge_rows = suppressed_df[suppressed_df["cluster_id"] == merge_cluster]
            if not merge_rows.empty:
                sentences_to_merge.extend(merge_rows.iloc[0]["sentences"])
        
        # Append sentences to the target cluster
        target_row = suppressed_df[suppressed_df["cluster_id"] == target_cluster]
        if not target_row.empty:
            target_row_index = target_row.index[0]
            suppressed_df.at[target_row_index, "sentences"] = (
                suppressed_df.at[target_row_index, "sentences"] + sentences_to_merge
            )
        
        # Remove merged clusters
        suppressed_df = suppressed_df[~suppressed_df["cluster_id"].isin(clusters_to_merge_into)]
    
    # Reset the index for a clean DataFrame
    suppressed_df.reset_index(drop=True, inplace=True)
    return suppressed_df


# Define clusters to suppress
clusters_to_suppress = [
    1, "11-0"
]

# Define clusters to merge
clusters_to_merge = {
    17: [18],
    38: [39]
}

# Apply suppression and merging
cleaned_cluster_df = suppress_and_merge_clusters(new_cluster_df, clusters_to_suppress, clusters_to_merge)

# Display the updated DataFrame
print(cleaned_cluster_df)

In [ ]:
def remove_duplicate_sentences(cluster_df):
    """
    Removes duplicate sentences within each row of the DataFrame.
    
    Parameters:
    - cluster_df: DataFrame containing cluster information with a 'sentences' column.

    Returns:
    - Updated DataFrame with unique sentences in each cluster.
    """
    # Ensure each row's "sentences" list contains only unique values
    cluster_df["sentences"] = cluster_df["sentences"].apply(lambda x: list(set(x)))
    return cluster_df


# Apply the function to remove duplicates
cleaned_cluster_df = remove_duplicate_sentences(cleaned_cluster_df)

# Display the updated DataFrame
print(cleaned_cluster_df)

In [ ]:
def assign_cluster_names(cluster_df, cluster_name_mapping):
    """
    Assign cluster names based on the provided mapping.

    Parameters:
    - cluster_df: DataFrame containing clusters.
    - cluster_name_mapping: Dictionary mapping cluster_id to cluster names.

    Returns:
    - Updated DataFrame with a new column for cluster names.
    """
    cluster_df = cluster_df.copy()  # Avoid modifying the original DataFrame

    # Assign names using the mapping
    cluster_df["Cluster Name"] = cluster_df["cluster_id"].map(cluster_name_mapping)

    # Fill missing names with "Unnamed Cluster"
    cluster_df["Cluster Name"].fillna("Unnamed Cluster", inplace=True)

    return cluster_df


# Define the cluster name mapping (shortened for brevity; use the full mapping provided)
cluster_name_mapping = {
    10: "Public or Private Investments",  
    14: "Carpooling"
}

# Apply the function to assign names
named_cluster_df = assign_cluster_names(cleaned_cluster_df, cluster_name_mapping)

# Display the updated DataFrame
print(named_cluster_df)

## 6. Export the policy clustering data

In [ ]:
# Update with your desired output path
named_cluster_df.to_csv(export_path, index=False)